## Travel Agent, Online MCP

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage

In [7]:
client = MultiServerMCPClient({
    "travel_server": {
        "transport": "streamable_http",
        "url": "https://mcp.kiwi.com"
    }
})


In [8]:
tools = await client.get_tools() 
print(tools)

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [13]:
agent = create_agent(
    "gpt-5-nano",
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)
config = {"configurable": {"thread_id": "1"}}

In [14]:
response = await agent.ainvoke(
    {"messages": [HumanMessage(
        content="Get me a direct flight from San Francisco to New York on March 1st 2026 in USD",
    )]},
    config=config
)

print(response)


{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to New York on March 1st 2026 in USD', additional_kwargs={}, response_metadata={}, id='8827341d-b4e0-4c03-a517-891b2418da87'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 639, 'prompt_tokens': 1235, 'total_tokens': 1874, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1024}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpQCn3q5Eq83y0H6fzPq2Pm4oCPWm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b43f2-1493-7272-a7c6-c4325ff3ead2-0', tool_calls=[{'name': 'search-flight', 'args': {'flyFrom': 'San Francisco', 'flyTo': 'New York', 'departureDate': '01/03/2026', 'passengers': {'adu

In [15]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to New York on March 1st 2026 in USD', additional_kwargs={}, response_metadata={}, id='8827341d-b4e0-4c03-a517-891b2418da87'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 639, 'prompt_tokens': 1235, 'total_tokens': 1874, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1024}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpQCn3q5Eq83y0H6fzPq2Pm4oCPWm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b43f2-1493-7272-a7c6-c4325ff3ead2-0', tool_calls=[{'name': 'search-flight', 'args': {'flyFrom': 'San Francisco', 'flyTo': 'New York', 'departureDate': '01/03/2026', 'pass

In [16]:
print(response["messages"][-1].content)


Here are direct flights from San Francisco (SFO) to New York (JFK or EWR) on March 1, 2026, in USD. I’ve grouped them by the cheapest options, then by shortest duration, then other direct options.

Cheapest options (price: 107 USD)
- Route: SFO → JFK
  - Times: 03/01 06:29 → 03/01 15:00 (5h 31m)
  - Cabin: Economy
  - Price: 107 USD
  - Booking: https://on.kiwi.com/Td5zJK

- Route: SFO → JFK
  - Times: 03/01 22:54 → 03/02 07:30 (5h 36m)
  - Cabin: Economy
  - Price: 107 USD
  - Booking: https://on.kiwi.com/AkD8Ib

Shortest duration (5h 30m)
- Route: SFO → EWR
  - Times: 03/01 08:00 → 03/01 16:30 (5h 30m)
  - Cabin: Economy
  - Price: 110 USD
  - Booking: https://on.kiwi.com/WUcmvk

- Route: SFO → EWR
  - Times: 03/01 16:00 → 03/01 21:30 (5h 30m)
  - Cabin: Economy
  - Price: 110 USD
  - Booking: https://on.kiwi.com/WUcmvk

Other direct options (higher prices)
- Route: SFO → JFK
  - Times: 03/01 09:00 → 03/01 17:36 (5h 36m)
  - Cabin: Economy
  - Price: 228 USD
  - Booking: https://on.k